### 数据预处理

In [1]:
stop_path = 'data/stopword.txt'
stopwords = open(stop_path).readlines()
stopwords = [word.strip() for word in stopwords]
len(stopwords)

767

In [2]:
data_path = 'data/射雕英雄传.txt'
docs = open(data_path).readlines()
docs = [doc.strip() for doc in docs if doc.strip() != '']

In [3]:
# 总行数
print(len(docs), sum([len(doc) for doc in docs]))
# 总字数
print(len(max(docs, key=len)), max(docs, key=len))

8611 892286
507 郭靖点头道：“对，幸亏大哥聪明，料到了这着，倘若是我，定是上了他们的大当。”周伯通摇头道：“说到聪明伶俐，天下又有谁及得上黄老邪的？只不知他用甚么法子，居然找到了一个跟他一般聪明的老婆。那时候黄家嫂子微微一笑，道：‘周大哥，你号称老顽童，人可不胡涂啊，你怕我刘备借荆州是不是？我就在这里坐着瞧瞧，看完了马上还你，也不用到天黑，你不放心，在旁边守着我就是。’“我听她这么说，就从怀里取出经书，递了给她。黄家嫂子接了，走到一株树下，坐在石上翻了起来。黄老邪见我神色之间总是有点提心吊胆，说道：‘老顽童，当世之间，有几个人的武功胜得过你我两人？’我道：‘胜得过你的未必有。胜过我的，连你在内，总有四五人罢！’黄老邪笑道：‘那你太捧我啦，东邪、西毒、南帝、北丐四个人，武功各有所长，谁也胜不了谁。欧阳锋既给你师哥破去了“蛤蟆功”’，那么十年之内，他是比兄弟要逊一筹的了。还有个铁掌水上飘裘千仞，听说武功也很了得，那次华山论剑他却没来，但他功夫再好，也未必真能出神入化。老顽童，你的武功兄弟决计不敢小看了，除了这几个人，武林中数到你是第一。咱俩联起手来，并世无人能敌。’我道：‘那自然！’黄老邪道：‘所以啊，你何必心神不定？


In [4]:
import jieba

all_docs = []
for doc in docs:
    result = []
    words = jieba.cut(doc)
    for word in words:
        if word != '' and word not in stopwords:
            result.append(word)
    all_docs.append(result)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.195 seconds.
Prefix dict has been built succesfully.


### 开始处理数据

In [5]:
from gensim import corpora, models, similarities

In [6]:
# 制作字典库
dictionary = corpora.Dictionary(all_docs)
len(dictionary.keys())

45209

In [7]:
# dictionary.save('data/dictionary') # 保存
# dictionary = Dictionary.load('data/dictionary') # 加载 

In [8]:
# 制作语料库
corpus = [dictionary.doc2bow(doc) for doc in all_docs]

In [9]:
# corpora.MmCorpus.serialize('data/corpuse', corpus) # 保存
# corpus = corpora.MmCorpus('data/corpuse') # 加载

In [10]:
##### 测试数据 #####

test = '射雕英雄传中谁的武功天下第一'

# 结巴 搜索引擎模式
test_list = [word for word in jieba.cut_for_search(test) if word not in stopwords]
test_vec = dictionary.doc2bow(test_list)

print(test_list)
print(test_vec)

['射雕', '英雄', '英雄传', '中', '武功', '天下', '第一', '天下第一']
[(61, 1), (282, 1), (1081, 1), (2673, 1), (4402, 1), (11505, 1), (15710, 1)]


## TF-IDF

In [11]:
tfidf = models.TfidfModel(corpus)

In [12]:
similarity = similarities.Similarity('Similarity-TFIDF-index',
                                     corpus, 
                                     num_features=len(dictionary.keys()),
                                     num_best=20)

In [13]:
# 测试文档 TF-IDF 值
test_tfidf = tfidf[test_vec]
test_tfidf

[(61, 0.1475027481247322),
 (282, 0.31460561035969264),
 (1081, 0.20059470585041056),
 (2673, 0.4598651730371035),
 (4402, 0.36642297536463514),
 (11505, 0.5739969129055862),
 (15710, 0.40473915419438267)]

In [14]:
result = similarity[test_tfidf]

for idx, confidence in result:
    print(idx, confidence, docs[idx])

1061 0.25669920444488525 第五回　弯弓射雕(1)
1172 0.25669920444488525 第五回　弯弓射雕(2)
3880 0.20211602747440338 郭靖涨红了脸，答道：“我想，王真人的武功既已天下第一，他再练得更强，仍也不过是天下第一。我还想，他到华山论剑，倒不是为了争天下第一的名头，而是要得这部《九阴真经》。他要得到经书，也不是为了要练其中的功夫，却是相救普天下的英雄豪杰，教他们免得互相所杀，大家不得好死。”
2880 0.18251502513885498 武功天下第一的王真人已经逝世，剩下我们四个大家半斤八两，各有所忌。
8377 0.18021109700202942 你上得华山来，妄想争那武功天下第一的荣号，莫说你武功未必能独魁群雄，纵然是当世无敌，天下英雄能服你这卖国好徒么？”
8433 0.17648662626743317 “天下英雄，唯使君与叫化啦。我见了你女儿，肚里的蛔虫就乱钻乱跳，馋涎水直流。咱们爽爽快快的马上动手，是你天下第一也好，是我第一也好，我只等吃蓉儿烧的好菜。”
796 0.16413959860801697 令郎更是英雄人物，老英雄怎么不提？”王罕笑道：“老汉死了之后，自然是他统领部众。但他怎比得上他的两个义兄？札木合足智多谋。铁木真更是刚勇无双，他是赤手空拳，自己打出来的天下。蒙古人中的好汉，哪一个不甘愿为他卖命？”完颜洪烈道：“难道老英雄的将士，便不及铁木真汗的部下吗？”
2745 0.1605529934167862 黄蓉笑道：“你老人家料事如神。你说我爹爹很厉害，是不是？”洪七公冷冷的道：“他当然厉害，可也不见得是天下第一。”黄蓉拍手道：“那么定是您第一啦。”
3253 0.16004645824432373 黄蓉道：”既然你老人家武功第一，那部经书该归您所有啊。”裘千仞道：”我也懒得跟人家争了。那东邪、西毒、南帝、北丐四人都是半斤八两，这些年来人人苦练，要争这大下第一的名头。二次华山论剑，热闹是有得看的。”黄蓉道：“还有二次华山论剑么？”裘千仞道：“二十五年一世啊。
8337 0.1580556035041809 黄蓉道：“你到华山来，想是要争那‘武功天下第一’的名号了？”郭靖道：“蓉儿，你怎么又来取笑？我是要向周大哥请教一个法子，怎生将已会的武功尽数忘却。”当下将这些日来

## LSI

In [15]:
lsi = models.LsiModel(corpus)

In [16]:
similarity_lsi = similarities.Similarity('Similarity-LSI-index',
                                       lsi[corpus],
                                       num_features=len(dictionary.keys()),
                                       num_best=20)

In [17]:
# 测试文档 TF-IDF 值
test_tfidf = tfidf[test_vec]
# 测试文档 LSI 值
test_lsi = lsi[test_tfidf]
test_lsi

[(0, 0.0387138706329267),
 (1, 0.043207390085800136),
 (2, 0.0005964415645817649),
 (3, -0.02120243315129773),
 (4, 0.028296269553269877),
 (5, 0.05158493783999565),
 (6, 0.009550009714388812),
 (7, -0.007182281666504438),
 (8, -0.04210066237991415),
 (9, 0.013162268060440411),
 (10, -0.006596726355419172),
 (11, 0.029307581568898552),
 (12, -0.027634481609234228),
 (13, -0.0017740006064673325),
 (14, 0.04742262120549117),
 (15, 0.07896018920549557),
 (16, -0.0883856617718533),
 (17, -0.04113209950183925),
 (18, -0.039759765750902794),
 (19, -0.002940726998300545),
 (20, 0.09678718490175262),
 (21, -0.023716039760858725),
 (22, -0.04366289039030378),
 (23, -0.01481412671066042),
 (24, 0.031051230338084584),
 (25, 0.06587990730136174),
 (26, -0.037254474744587145),
 (27, 0.019927409763202522),
 (28, -0.011375309799679677),
 (29, -0.02541861265946013),
 (30, 0.0170720359472053),
 (31, -0.03473011658849206),
 (32, -0.012639490862995298),
 (33, 0.028806973439611396),
 (34, 0.04328635273189

In [18]:
result = similarity_lsi[test_lsi]

for idx, confidence in result:
    print(idx, confidence, docs[idx])

2880 0.6983173489570618 武功天下第一的王真人已经逝世，剩下我们四个大家半斤八两，各有所忌。
8377 0.642870306968689 你上得华山来，妄想争那武功天下第一的荣号，莫说你武功未必能独魁群雄，纵然是当世无敌，天下英雄能服你这卖国好徒么？”
7965 0.6409813165664673 两人回到帐中，这番当真研习起《九阴真经》上的武功来，谈论之下，均觉对方一年来武功大有长进，均感欣慰。
4332 0.61060631275177 他牢牢记住师兄王重阳的遗训，决不敢修习经中所载武功，哪知为了教导郭靖，每日里口中解释、手上比划，不知不觉的已把经文深印脑中，睡梦之间，竟然意与神会，奇功自成，这时把拳脚施展出来，却是无不与经中所载的拳理法门相合。他武功深湛，武学上的悟心又是极高，兼之《九阴真经》中所载纯是道家之学，与他毕生所学本是一理相通，他不想学武功，武功却自行扑上身来。他纵声大叫：“糟了，糟了，这叫做惹鬼上身，挥之不去了。
3222 0.5777295231819153 陆庄主虽没见过裘千仞的武功，但素仰他的威名，知道当年东邪、西毒、南帝、北丐、中神通五人在华山绝顶论剑，也曾邀他到场，只是他适有要事，未能赴约，但既受到邀请，自是武功卓绝，非同小可，纵使不及王重阳等五人，谅亦相差不远，有他在这里，黑风双煞是不能为恶的了，当下向郭靖及黄蓉道：“两位还没走，真好极了。这位裘老前辈武功极高，常人难以望其项背，天幸今日凑巧到来，我还忌惮甚么对头？待会两位请自行在卧室中休息，只要别出房门，那就没事。”
5277 0.5764594674110413 武术中有言道：”未学打人，先学挨打。”初练粗浅功夫，却须由师父传授怎生挨打而不受重伤，到了武功精深之时，就得研习护身保命、解穴救伤、接骨疗毒诸般法门。须知强中更有强中手，任你武功盖世，也难保没失手的日子。这《九阴真经》中的“疗伤篇”，讲的是若为高手以气功击伤，如何以气功调理真元，治疗内伤。至于折骨、金创等外伤的治疗，研习真经之人自也不用再学。
2166 0.5628796815872192 原来这参仙老怪本是长白山中的参客，后来害死了一个身受重伤的前辈异人，从他衣囊中得了一本武学秘本和十余张药方，照法修练研习，自此武功了得，兼而精通药理。药方中有一方是以药养蛇、从而易筋壮体的秘诀。
1232 0.5